In [1]:
print("Hello World!")

Hello World!


In [2]:
import fitz
import glob

def load_all_pdfs_in_directory(directory="./content/"):
  combined_text = ""
  pdf_paths = glob.glob(f"{directory}/*.pdf")

  for file_path in pdf_paths:
    with fitz.open(file_path) as pdf:
      for page in pdf:
        combined_text += page.get_text()

  return combined_text

pdf_data = load_all_pdfs_in_directory()
print("Pdf_data: ", pdf_data[:500])

Pdf_data:  Maruti Mohit Rayalacheruvu 
(857) 313-2407 | rayalacheruvu.m@northeastern.edu | https://linkedin.com/in/marutimohitr 
 
EDUCATION 
Master of Science in Information Systems, Northeastern University​
December 2024 
 
Bachelor of Engineering in Computer Science, Visvesvaraya Technological University​
July 2021 
 
TECHNICAL SKILLS 
Programming Languages: Python, Java, C#, C,  C++ 
Data Science & ML: Scikit-learn, Statistical Analysis, Data Visualization (Matplotlib, Seaborn), Classification Models, 


In [3]:
len(pdf_data)

13088

In [4]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1200,
    chunk_overlap=300
)

documents = text_splitter.split_text(pdf_data)
print("Number of documents: ", len(documents))


Number of documents:  15


In [5]:
documents[0]

'Maruti Mohit Rayalacheruvu \n(857) 313-2407 | rayalacheruvu.m@northeastern.edu | https://linkedin.com/in/marutimohitr \n \nEDUCATION \nMaster of Science in Information Systems, Northeastern University\u200b\nDecember 2024 \n \nBachelor of Engineering in Computer Science, Visvesvaraya Technological University\u200b\nJuly 2021 \n \nTECHNICAL SKILLS \nProgramming Languages: Python, Java, C#, C,  C++ \nData Science & ML: Scikit-learn, Statistical Analysis, Data Visualization (Matplotlib, Seaborn), Classification Models, \nFeature Engineering, Pandas, NumPy, Model Optimization, Jupyter, Predictive Modeling \nTechnologies: SQL (PostgreSQL), AWS, JavaScript, TypeScript, React, Node.js, Express, MongoDB, Docker, CI/CD \nFrameworks: Data Structures & Algorithms, Microservices Architecture, .NET Core, Git, Linux, Unit Testing, Postman \n \nWORK EXPERIENCE \nAssociate Software Engineer, Conga\u200b\n\u200b\nJuly 2021 – August 2022 \n●\u200b Architected cloud-native microservices with C#, .NET Co

In [6]:
from langchain_ollama import OllamaEmbeddings

embeddings = OllamaEmbeddings(model="llama3.2")

embedded_documents = embeddings.embed_documents(documents)

In [7]:
print("Embedding for the first document")
for i, value in enumerate(embedded_documents[0]):
  print(f"Component {i+1}: {value}")
  if i >= 5:
    break

Embedding for the first document
Component 1: 0.008276723
Component 2: -0.00096380425
Component 3: -0.01307932
Component 4: 0.008557427
Component 5: -0.006538015
Component 6: 0.013165651


In [8]:
from langchain_chroma import Chroma

# 4️⃣ Store Embeddings in ChromaDB
chroma_db = Chroma.from_texts(texts=documents, embedding=embeddings, persist_directory="chroma_db")

# 5️⃣ Load ChromaDB Index
vectorstore = Chroma(persist_directory="chroma_db", embedding_function=embeddings)

# 6️⃣ Create User Chat History Vectorstore
history_db = Chroma(persist_directory="history_db", embedding_function=embeddings)

# 7️⃣ Define Retriever
top_k = 6  # Number of top results to retrieve
doc_retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": top_k})
history_retriever = history_db.as_retriever(search_type="similarity", search_kwargs={"k": top_k})

In [9]:
print(f"Number of documents indexed: {len(documents)}")

Number of documents indexed: 15


In [31]:
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 6})

In [10]:
def pretty_print(docs):
  print(
      f"\n{'-' * 100}\n".join(
          [f"Document {i+1}:\n\n"+ d.page_content for i,d in enumerate(docs)]
      )
  )

In [11]:
import os
os.environ["COHERE_API_KEY"] = "fqbB6WXKvozEcRdd0nYShpVATojBVadM4EmCUs2G"  # Set the API key
print(f"COHERE_API_KEY: {os.getenv('COHERE_API_KEY')}")  # Make sure the key is not None or empty


COHERE_API_KEY: fqbB6WXKvozEcRdd0nYShpVATojBVadM4EmCUs2G


In [13]:
from langchain.embeddings import OllamaEmbeddings
from langchain_chroma import Chroma
import chromadb
from langchain_cohere import CohereRerank
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
import os
import cohere

cohere_api_key = os.getenv("COHERE_API_KEY")
cohere_client = cohere.ClientV2(cohere_api_key)

cohere_reranker = CohereRerank(client=cohere_client, model="rerank-english-v3.0", top_n=3)

# compression_retriever = ContextualCompressionRetriever(
#     base_compressor=cohere_reranker,
#     base_retriever=retriever
# )

doc_compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_reranker,
    base_retriever=doc_retriever
)
history_compression_retriever = ContextualCompressionRetriever(
    base_compressor=cohere_reranker,
    base_retriever=history_retriever
)

In [14]:
from langchain.chains import RetrievalQA
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.2", temperature=0.5, max_tokens=3000)
# qa_chain = RetrievalQA.from_chain_type(llm, retriever=compression_retriever)

In [15]:
# user_query_1 = "who does the document discuss about?"
# user_query_2 = "what are their achievements?"
# user_query_3 = "what certifications does mahith have? "
# user_query_4 = "what certifications does maruti have?"
# user_query_5 = "Does maruti have any experience working with AI/ML or Blockchain?"
# user_query_7 = "what are their emails?"
# user_query_8 = "my name is mahith"
# user_query_9 = "my roommate name is ankit"
# user_query_10 = "whats my name ? who's my roommate? "

# user_queries = [user_query_1, user_query_2, user_query_3, user_query_4, user_query_5, user_query_7, user_query_8, user_query_9, user_query_10]

In [20]:
# 🔟 Interactive User Query Loop
while True:
    user_query = input("Ask a question (or type 'exit' to quit): ")
    if user_query.lower() == 'exit':
        print("Goodbye!")
        break
    
    # Retrieve relevant information from document and history databases
    doc_results = doc_compression_retriever.get_relevant_documents(user_query)
    history_results = history_compression_retriever.get_relevant_documents(user_query)
    
    doc_context = "\n".join([doc.page_content for doc in doc_results])
    history_context = "\n".join([hist.page_content for hist in history_results])
    
    prompt = (
        "You are an expert assistant with access to two sources of knowledge: "
        "document knowledge and chat history. "
        "Use document knowledge to answer factual questions and chat history to recall user-provided context.\n\n"
        "### Document Knowledge:\n"
        f"{doc_context}\n\n"
        "### Chat History:\n"
        f"{history_context}\n\n"
        "Answer the user's query using the most relevant information from the provided sources. "
        f"If relevant information is missing, state that clearly.\n\nUser Query: {user_query}"
    )
    
    response = llm.invoke(prompt)
    response_text = response.content if hasattr(response, 'content') else str(response)
    print(f"Answer: {response_text}\n")
    
    # Store user query and response in history_db
    history_db.add_texts(user_query)

Ask a question (or type 'exit' to quit):  hello


Number of requested results 6 is greater than number of elements in index 5, updating n_results = 5


Answer: Unfortunately, I don't have enough context to provide a meaningful response to your greeting. The chat history is currently empty, and there are no documents or other knowledge sources available to draw upon. Could you please provide more context or ask a specific question? I'll do my best to assist you.



Ask a question (or type 'exit' to quit):  what does the documents discuss about?


Answer: The document knowledge discusses various topics:

1. Object Detection on Google’s Open Images Dataset: This section talks about using distributed computing capabilities to preprocess a large dataset and implementing object detection models.
2. Game of Tigers and Goats: It discusses developing and implementing a game that features two players (Goat and Tiger) and training SARSA reinforcement learning agents to optimize player strategies.
3. Parkinson’s Disease Regression Analysis: This section covers statistical tests, feature selection techniques, and regression models for predicting Total UPDRS scores in Parkinson's patients using voice-related features.

These topics seem to be unrelated to each other, as they cover different domains such as computer vision, game development, and medical research.



...

Ask a question (or type 'exit' to quit):  my name is mahith and my roommmate name is ankit


Answer: I don't have any information about a person named Mahith having a roommate named Ankit in our chat history or document knowledge. Could you please provide more context or clarify who Mahith is? I'll do my best to help.



Ask a question (or type 'exit' to quit):  whats my name


Answer: I don't have any information about your name in either the document knowledge or chat history. Could you please provide more context or clarify who you are asking about? I'll do my best to help based on the available information.



KeyboardInterrupt: Interrupted by user